In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np

from gammapy.data import EventList
from gammapy.datasets import MapDataset
from gammapy.irf import PSFMap, EDispKernelMap
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
    TemplateSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    create_fermi_isotropic_diffuse_model,
)
from gammapy.modeling import Fit

In [6]:
dataset_names = ['low_energy', 'med_energy', 'hi_energy']
main_path = '/home/woody/caph/mppi062h/woody_output/final/fermilat/'

N_bin_met = 8
radius_list = [12, 5, 5]

In [3]:
# these values comes from the initial choices of separating the data
bin_per_dec= [12, 12, 8]
dataset_e_edges = [100, 600, 4e3, 1e6] #u.MeV

low_energy_bins = np.concatenate((np.logspace(2,4,12)[np.logspace(2,4,12)<=4e3], [4e3]))
full_energy_bins = np.concatenate((low_energy_bins, np.logspace(3,6,8)[np.logspace(3,6,8)>=4e3]))
full_energy_bins = np.insert(full_energy_bins, np.sum(full_energy_bins<6e2), 6e2)
print(full_energy_bins)

[1.00000000e+02 1.51991108e+02 2.31012970e+02 3.51119173e+02
 5.33669923e+02 6.00000000e+02 8.11130831e+02 1.23284674e+03
 1.87381742e+03 2.84803587e+03 4.00000000e+03 7.19685673e+03
 1.93069773e+04 5.17947468e+04 1.38949549e+05 3.72759372e+05
 1.00000000e+06]


In [11]:
def from_folder(folder, dataset_idx):
    gc_pos = SkyCoord(0, 0, unit="deg", frame="galactic")
    energy_axis = MapAxis.from_edges(full_energy_bins, name="energy", unit="MeV", interp="log")

    events = EventList.read(f'{folder}/ft1_00.fits')
    counts = Map.create(
        skydir=gc_pos,
        width=2*radius_list[dataset_idx]*u.deg,
        proj="CAR",
        frame="galactic",
        binsz=0.08,
        axes=[energy_axis],
        dtype=float,
    )
    counts.fill_events(events)

    exposure_hpx = Map.read(f'{folder}/bexpmap_00.fits')
    axis = MapAxis.from_nodes(counts.geom.axes[0].center, name="energy_true", unit="MeV", interp="log")
    geom = WcsGeom(wcs=counts.geom.wcs, npix=counts.geom.npix, axes=[axis])
    exposure = exposure_hpx.interp_to_geom(geom)

    psf = PSFMap.read(
        f"{folder}/psf_00.fits", format="gtpsf"
    )
    psf_kernel = psf.get_psf_kernel(
        position=geom.center_skydir, geom=geom, max_radius="1 deg"
    )
    
    e_true = exposure.geom.axes["energy_true"]
    edisp = EDispKernelMap.from_diagonal_response(
    energy_axis_true=e_true, energy_axis=energy_axis)

    dataset = MapDataset(counts=counts, exposure=exposure, psf=psf, edisp=edisp)
    dataset.write(f'{folder}/fermi_dataset.fits', overwrite=True)

In [12]:
for dataset_idx, dataset_name in enumerate(dataset_names):
    folder = f'{main_path}/{dataset_name}/full_time'
    #from_folder(folder, dataset_idx)
    if dataset_idx != 0:
        for n_bin_met in range(N_bin_met-1):
            folder = f'{main_path}/{dataset_name}/{n_bin_met}_bin_met'
            from_folder(folder, dataset_idx)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 54766.757837 from DATE-OBS.
Set MJD-END to 55496.757801 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55496.757801 from DATE-OBS.
Set MJD-END to 56227.757801 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 56227.757801 from DATE-OBS.
Set MJD-END to 56957.757801 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 56957.757801 from DATE-OBS.
Set MJD-END to 57688.757801 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 57688.757801 from DATE-OBS.
Set MJD-END to 58418.757801 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 58418.757801 from DATE-OBS.
Set MJD-END to 59149.743